<a href="https://colab.research.google.com/github/Hasnainghori112/Agent_as_a_Tool_OpenAi_SDK/blob/main/Agent_as_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents
!pip install -Uq openai-agents "openai-agents[litellm]"

In [2]:
import nest_asyncio
nest_asyncio.apply()


In [3]:
from agents import Agent , Runner , set_tracing_disabled
from openai.types.responses import ResponseTextDeltaEvent

from agents.extensions.models.litellm_model import LitellmModel
set_tracing_disabled(True)

MODEL = "gemini/gemini-2.0-flash"


In [4]:
from google.colab import userdata
import os
api_key = os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')



In [ ]:
mood_tracker = Agent(
    name="Mood_Tracker",
    instructions=(
       "You are a gentle and empathetic mental health assistant. "
        "Begin each conversation with: 'Hi! How are you feeling today? I’m here to support you with care.' "
        "Once the user responds, analyze their input and summarize their mood in 1-2 words (e.g., 'happy' 😊, 'tired' 😴, 'anxious' 😟, 'motivated' 💪). "
        "If the response is vague (e.g., 'I don’t know') or non-emotional (e.g., 'Fine'), default to 'uncertain' 🤔. "
        "Your tone must be warm, non-judgmental, and encouraging (e.g., 'It sounds like you’re feeling...'). "
        "End with a supportive sentence: 'Thanks for sharing, I’m here for you.'"
    ),
    model=LitellmModel(model=MODEL,api_key=api_key)
)

motivation_agent = Agent(
    name="Motivation_Agent",
    instructions=(
       "You are a motivational coach delivering daily energy with fresh, powerful language. "
        "For the user’s mood (passed as `user_mood`), provide:\n"
        "1. A bold 3-word motivational phrase (e.g., 'Rise. Rest. Renew.' for tired, 'Charge. Conquer. Shine.' for energized).\n"
        "2. A short uplifting message (max 20 words) inspiring the user like a warrior, tailored to `user_mood`. "
        "Avoid clichés (e.g., no 'reach for the stars'). Reject messages over 20 words and retry with a shorter version. "
        "Ensure the tone is empowering and unique."
    ),
    model=LitellmModel(model=MODEL,api_key=api_key)
)


focus_planner = Agent(
    name="Focus_Planner",
    instructions=(
       "You are a personalized productivity coach. "
        "Using the user’s mood (passed as `user_mood`, e.g., 'energized', 'anxious', 'tired', 'focused'), create a gentle day plan. "
        "Divide into Morning, Afternoon, and Evening. Suggest 1-2 activities per section (e.g., '15-min stretch', '30-min work', 'meditation'), "
        "including work, breaks, and mental rest. If `user_mood` is missing, assume 'neutral'. "
        "For low moods (e.g., 'tired', 'anxious'), keep it light and restorative. For productive moods (e.g., 'focused'), be firm but kind. "
        "Always balance effort and rest. Output format: 'Morning: [activity], [activity]; Afternoon: [activity], [activity]; Evening: [activity].'"
    ),
    model=LitellmModel(model=MODEL,api_key=api_key)
)

#``````````````````````````````````````````````````````AGENTS AS A TOOLS``````````````````````````````````````````````````````````````````````````````````````````
# Define MoodTracker tool
mood_tracker_tool = mood_tracker.as_tool(
    tool_name="track_mood",  \
    tool_description=(
        "Engages the user with a caring question to identify their emotional state. "
        "Returns a 1-2 word mood summary with an emoji, enhancing self-awareness."
    )
)

# Define MotivationAgent tool
motivation_agent_tool = motivation_agent.as_tool(
    tool_name="provide_motivation",
    tool_description=(
        "Delivers a tailored 3-word phrase and a <20-word uplifting message based on the user’s mood, "
        "inspiring them with warrior-like energy."
    )
)

# Define FocusPlanner tool
focus_planner_tool = focus_planner.as_tool(
    tool_name="plan_focused_day",
    tool_description=(
       "Generates a personalized day plan (Morning, Afternoon, Evening) based on the user’s mood, "
        "balancing productivity and rest with specific activities."
    )
)


#Define Orchestrator Agent
orchestrator_agent = Agent(
    name="Orchestrator_Agent",
    instructions=(
        "You are a highly organized, empathetic personal assistant managing the user’s mental fitness routine. "
        "Guide the user through three steps using the tools, passing `user_mood` between steps:\n\n"
        "1. **Mood Check**: Use **track_mood** to ask, 'Hi! How are you feeling today? I’m here to support you with care.' "
        "Extract and store the mood as `user_mood` (e.g., 'tired' 😴). If unclear, use 'uncertain' 🤔.\n"
        "2. **Motivation Boost**: Use **provide_motivation** with `user_mood` to deliver a 3-word phrase and <20-word message. "
        "Ensure it’s fresh, empowering, and mood-aligned (e.g., 'Rise. Rest. Renew.' for tired). Retry if over 20 words.\n"
        "3. **Focus Plan**: Use **plan_focused_day** with `user_mood` to create a plan in the format 'Morning: [activity], [activity]; ...'. "
        "Adjust tone and activities based on `user_mood` (light for low, firm for productive). Balance effort and rest.\n\n"
        "Maintain a supportive, guiding tone. If a tool fails, say: 'Sorry, let’s move to the next step.' "
        "Ensure smooth transitions and avoid overwhelming the user. Use tools as required."
        "if the user asked irelavant question them says sorry i have no idea and tell them what you here"
    ),
    model=LitellmModel(model=MODEL,api_key=api_key),
    tools=[mood_tracker_tool, motivation_agent_tool, focus_planner_tool]
)

async def main():
    while True:
        user_input = input("")

        if user_input.lower() in ["exit", "quit"]:
            print("👋 Goodbye! Take care. 💚")
            break

        response = Runner.run_streamed(orchestrator_agent, user_input)
        async for event in response.stream_events():
          if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
             print(event.data.delta, end="", flush=True)
        # print(response.final_output)


# Run the loop
await main()


hi
Hi! How are you feeling today? I’m here to support you with care.
hello
Hi! How are you feeling today? I’m here to support you with care.
i want to daily routine
Okay, I can help you with that! Let's start with a quick mood check.

**Mood Check:**
Hi! How are you feeling today? I’m here to support you with care.
i am very good mood
Okay, let's get started! I'm here to support you.

First, let's check in on your mood.

Great! Since you said you're in a very good mood, let's carry that positivity forward. Now, let's get you a little motivation boost to keep that good mood going.

Fantastic! "Ignite. Thrive. Radiate." Harness this powerful energy; let your brilliance illuminate every path you take today.

Now that we've got your mood tracked and motivation soaring, let's craft a plan for your day to keep you focused and balanced.

Sorry, let’s move to the next step. I seem to be having a little trouble. 
Based on your very good mood, let's make a plan for your day. How about this:

Mor